# Reading the geospatial data

In [1]:
import numpy as np
import pandas as pd
import os

geo = pd.read_csv(
    "/Users/skronski/Documents/Projects/e_commerce/01_input/olist_geolocation_dataset.csv",
    dtype={"geolocation_zip_code_prefix": str},
)

# Gets the first three and four first digits of zip codes, we will explore this further to understand how zip codes works
geo["geolocation_zip_code_prefix_1_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:1
]
geo["geolocation_zip_code_prefix_2_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:2
]
geo["geolocation_zip_code_prefix_3_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:3
]
geo["geolocation_zip_code_prefix_4_digits"] = geo["geolocation_zip_code_prefix"].str[
    0:4
]
geo.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,geolocation_zip_code_prefix_1_digits,geolocation_zip_code_prefix_2_digits,geolocation_zip_code_prefix_3_digits,geolocation_zip_code_prefix_4_digits
0,01037,-23.545621,-46.639292,sao paulo,SP,0,01,010,0103
1,01046,-23.546081,-46.644820,sao paulo,SP,0,01,010,0104
2,01046,-23.546129,-46.642951,sao paulo,SP,0,01,010,0104


In [4]:
# plot wtih holoviews + datashader - bokeh with map background
import holoviews as hv
import geoviews as gv
import datashader as ds
from colorcet import fire, rainbow, bgy, bjy, bkr, kb, kr
from datashader.colors import colormap_select, Greys9
from holoviews.streams import RangeXY
from holoviews.operation.datashader import datashade, dynspread, rasterize
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
hv.extension('bokeh')

%opts Overlay [width=800 height=600 toolbar='above' xaxis=None yaxis=None]
%opts QuadMesh [tools=['hover'] colorbar=True] (alpha=0 hover_alpha=0.2)

T = 0.05
PX = 1

def plot_map(data, label, agg_data, agg_name, cmap):
    url="http://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{Z}/{Y}/{X}.png"
    geomap = gv.WMTS(url)
    points = hv.Points(gv.Dataset(data, kdims=['x', 'y'], vdims=[agg_name]))
    agg = datashade(points, element_type=gv.Image, aggregator=agg_data, cmap=cmap)
    zip_codes = dynspread(agg, threshold=T, max_px=PX)
    hover = hv.util.Dynamic(rasterize(points, aggregator=agg_data, width=50, height=25, streams=[RangeXY]), operation=hv.QuadMesh)
    hover = hover.options(cmap=cmap)
    img = geomap * zip_codes * hover
    img = img.relabel(label)
    return img

ModuleNotFoundError: No module named 'geoviews'